In [17]:
import pandas as pd
import os
from pandas import ExcelWriter
from pandas import ExcelFile
import json

filename="instance-name"

excel_data = pd.read_excel(f"./{filename}.xlsx", sheet_name=None)

In [18]:
inputs = excel_data["Inputs"].drop(labels=["Description", "or MQTT params"], axis=1)
outputs = excel_data["Outputs"].drop(labels=["Description", "MQTT params"], axis=1)

start_config = excel_data["Start_config"].drop(labels=["Description"], axis=1)
start_config_dict = start_config["Value"].to_dict()

In [19]:
start_config_dict

{'control_frequency': 30,
 'horizon_in_steps': 48,
 'dT_in_seconds': 3600,
 'model_name': 'testModel1',
 'repetition': 1,
 'solver': 'ipopt'}

In [20]:
import xlsxwriter as xl

sheets = ['inputs', 'outputs', 'start']



col_headers = {'inputs': ['Input_name', 'value', 'or filename', '', 'or MQTT params', 'Description'], 'outputs': ['Output_name', '', 'MQTT params', 'Description'], 'start': ['configs', 'Value', 'Description']} 

default_cell_values = {'inputs': [['','','url','',''],['','','topic','',''],['','','qos','','']], 'outputs': [['url','',''],['topic','',''],['qos','','']], 'start': [['','']] }



def generate_xlsx_instance_config(data, filepath):
        """Generate excel sheet for instance configuration
        
        Arguments:
            data {dict} -- Input data with list of config to be filled
            filepath {str} -- File path of the excel file to be created
        """
        if os.path.isfile(filepath):
            print("File already exists. Cannot overwrite file")
        else:

            with xl.Workbook(filepath) as workbook:

                row_header_format = workbook.add_format({'align': 'center', 'bold': True, 'valign': 'vcenter'})
                col_header_format = workbook.add_format({'align': 'center', 'bold': True, 'valign': 'vcenter'})    
                cell_format = workbook.add_format()

                # Formatting information
                BORDER_SIZE = 2
                COL_HEADER_WIDTH = 30
                NORMAL_COL_WIDTH = 20
                SHORT_COL_WIDTH = 5

                row_header_format.set_border(BORDER_SIZE)
                col_header_format.set_border(BORDER_SIZE)
                cell_format.set_border(BORDER_SIZE)

                # Generate worksheets and format
                for worksheet_name in sheets:
                    sheet = workbook.add_worksheet(worksheet_name)

                    # Create column headers
                    col_header = col_headers[worksheet_name]
                    for col_num, col_value in enumerate(col_header):
                        sheet.write(0, col_num, col_value, col_header_format)

                        # Column formatting
                        sheet.set_column(col_num, col_num, NORMAL_COL_WIDTH)

                        # Widen first column
                        if col_num == 0:
                            sheet.set_column(col_num, col_num, COL_HEADER_WIDTH)
                        # Shorten empty column
                        if col_value == "":
                            sheet.set_column(col_num, col_num, SHORT_COL_WIDTH)


                    # Create row headers and format rows based on default inputs
                    row_header = data[worksheet_name]
                    cell_values = default_cell_values[worksheet_name]
                    for row_num, row_value in enumerate(row_header):

                        # Different formatting for start config sheet
                        if worksheet_name == "start":
                            sheet.write(row_num + 1, 0, row_value, cell_format)

                            for cell_row_num, cell_row in enumerate(cell_values):
                                for cell_col_num, cell_value in enumerate(cell_row):
                                    sheet.write(cell_row_num + row_num + 1, cell_col_num + 1, cell_value, cell_format)
                            # skip the rest of the code for start config
                            continue

                        # Merge every three rows for 
                        first_row = 1 + (row_num * 3)
                        last_row = first_row + 2
                        sheet.merge_range(first_row, 0, last_row, 0, row_value, row_header_format)

                        for cell_row_num, cell_row in enumerate(cell_values):
                            for cell_col_num, cell_value in enumerate(cell_row):
                                # Get col name and set formatting per row or per merged range
                                col_header_name = col_header[cell_col_num+1]
                                if "MQTT" in col_header_name or col_header_name == "":
                                    sheet.write(cell_row_num + first_row, cell_col_num + 1, cell_value, cell_format)
                                else:
                                    sheet.merge_range(first_row, cell_col_num + 1, last_row, cell_col_num + 1, "", cell_format)

In [22]:
data = {'inputs': [ 'ESS_Capacity', 'ESS_Max_Charge_Power', 'ESS_Max_Discharge_Power', 'P_Grid_Max_Export_Power', 'P_PV', 'PV_Inv_Max_Power', 'P_Load'], 'outputs': ['P_Grid_Output', 'P_PV_Output', 'P_ESS_Output', 'SoC_ESS'], 'start': ['control_frequency', 'horizon_in_steps', 'dT_in_seconds', 'model_name', 'repetition', 'solver']} 
filepath="../notebooks/new.xlsx"

generate_xlsx_instance_config(data, filepath)

File already exists. Cannot overwrite file
